Este colab fue desarrollado por Arnold Charry Armero.

# Fuerza Policial Smalltown

Durante cada período de 4 horas, la fuerza policial de Smalltown
requiere el siguiente número de policías de servicio: 12
medianoche a 4 a.m.—8; 4 a 8 a.m.—7; 8 a.m. a las 12 del mediodía—
6; 12 del mediodía a 4 p.m.—6; 4 a 8 p.m.—5; 8 p.m. a 12
medianoche—4. Cada policía trabaja dos veces consecutivas.
Turnos de 4 horas. Formule un PL que pueda usarse para minimizar
el número de agentes de policía necesarios para cumplir con los requisitos de Smalltown
requerimientos diarios.

## Conjuntos

$ i = \text{Policías que comienzan en la franja (hora) } i$

$ j = \text{Policías que comienzan en la franja (hora) } j$

## Parámetros

$ P_{ij} = \textrm{1 si el policia } i \textrm{ se encuentra disponible en el momento } j \textrm{, 0 lo contrario.}$

$ D_{j} = \textrm{Números de policias } j \textrm{ necesarios}$

## Variables de decisión

$ x_{i} = \textrm{Policias que comienzan su empleo en la franja } i $

## Función objetivo

$$ \textrm{Min } z = \sum_{i = 1}^{n}x_{i} $$

## Restricciones

Se debe satisfacer la demanda $j$,

$$ \sum_{i = 1}^{n}x_{i} P_{ij} \geq D_{j}, \forall i$$

In [7]:
#Se instala la libreria
!pip install pulp

In [8]:
#Se importa la libreria
from pulp import *

In [9]:
#Conjuntos
dias1 = range(6)
dias2 = range(6)

In [10]:
D = [8, 7, 6, 6, 5, 4]

# Número de días en la semana
num_dias = 6

def crear_matriz_turnos(num_dias):
    # Crear una matriz de ceros
    matriz_turnos = [[0 for _ in range(num_dias)] for _ in range(num_dias)]

    # Asignar turnos de trabajo (5 días trabajados, 2 días de descanso)
    for i in range(num_dias):
        # Determinar el inicio del ciclo de trabajo del empleado
        inicio = i % num_dias
        for dia in range(num_dias):
            if (dia >= inicio and dia < inicio + 2) or (inicio + 2 > num_dias and dia < (inicio + 2) % num_dias):
                matriz_turnos[i][dia] = 1

    return matriz_turnos

# Llamar a la función para crear la matriz de turnos
P = crear_matriz_turnos(num_dias)

# Mostrar la matriz
print("---Matriz de Turnos---")
for i in range(num_dias):
    for j in range(num_dias):
        print(P[i][j], end="  " if P[i][j] >= 0 and P[i][j] < 10 else " ")
    print("")

---Matriz de Turnos---
1  1  0  0  0  0  
0  1  1  0  0  0  
0  0  1  1  0  0  
0  0  0  1  1  0  
0  0  0  0  1  1  
1  0  0  0  0  1  


In [14]:
# Crear la instancia del modelo
model = LpProblem(name="Fuerza_Policial_Smalltown", sense=LpMinimize)

# Inicialización de las variables de decisión
X = LpVariable.dicts("X", (dias1), 0, None, LpInteger)

#Función objetivo
model += lpSum([X[i] for i in dias1])

# Restricciones
for j in dias2:
    model += lpSum([X[i] * P[i][j] for i in dias1]) >= D[j]

# Resolver el problema
model.solve()

# Imprimir el estado del problema
print("Status:", LpStatus[model.status])

# Imprimir las variables
for v in model.variables():
    print(v.name, "=", v.varValue)

# Imprimir la función objetivo
print("Trabajadores contratados =", value(model.objective))

Status: Optimal
X_0 = 2.0
X_1 = 5.0
X_2 = 1.0
X_3 = 5.0
X_4 = 0.0
X_5 = 6.0
Trabajadores contratados = 19.0


In [12]:
model

Fuerza_Policial_Smalltown:
MINIMIZE
1*X_0 + 1*X_1 + 1*X_2 + 1*X_3 + 1*X_4 + 1*X_5 + 0
SUBJECT TO
_C1: X_0 + X_5 >= 8

_C2: X_0 + X_1 >= 7

_C3: X_1 + X_2 >= 6

_C4: X_2 + X_3 >= 6

_C5: X_3 + X_4 >= 5

_C6: X_4 + X_5 >= 4

VARIABLES
0 <= X_0 Integer
0 <= X_1 Integer
0 <= X_2 Integer
0 <= X_3 Integer
0 <= X_4 Integer
0 <= X_5 Integer